# Spark-HPCC Connector for HPCC Systems Platform and Spark Connectivity

This example demonstrates how to use the Spark-HPCC Connector to read and write data from / to HPCC Systems clusters, as well as providing basic setup information for the Spark-HPCC connector.

## Spark-HPCC Connector Installation:

---

The Spark-HPCC Connector jar and its dependencies need to be made available to all Spark worker nodes and the Spark driver application. This can be done by adding the Spark-HPCC connector jar to the classpath on every node in the Spark cluster and to the classpath for the Spark driver, or by using the ```--jars``` option when executing spark-submit or pyspark.

Download the Spark-HPCC jar with dependencies from Maven Central: https://mvnrepository.com/artifact/org.hpccsystems/spark-hpcc

### Example of using the jars option:
```
pyspark --jars spark-hpcc-9.2.2-1-jar-with-dependencies.jar
```

### Adding Spark-HPCC jar to classpath
The Spark-HPCC jar can also be added to the classpath through various means depending on the configuration of your Spark cluster, more information about updating the classpath can be found within the Spark documentation: https://spark.apache.org/docs/latest/configuration.html

# Creating a test dataset

The following code will create a dataframe with two columns, key and fill, that will be used to demonstrate the reading and writing functionality of the Spark-HPCC connector.

---

In [1]:
from pyspark.sql import SparkSession
import random

In [2]:
data = [(i, int(1e10 * random.random())) for i in range(1000)]
df = spark.createDataFrame(data, ["key", "fill"])
df.show()

+---+----------+
|key|      fill|
+---+----------+
|  0|8802828259|
|  1|6769815762|
|  2|3802883209|
|  3|7410875502|
|  4| 135941891|
|  5|2166775522|
|  6| 883758744|
|  7|3540920239|
|  8|  20735769|
|  9|4435630331|
| 10|2674809260|
| 11|2527618309|
| 12|9667830781|
| 13|8674197252|
| 14|2973016826|
| 15|7810363123|
| 16|3267854158|
| 17|9341494262|
| 18|8715303091|
| 19|7096204106|
+---+----------+
only showing top 20 rows



# Writing Data to HPCC Systems

---

A Spark Dataframe can be written to HPCC using the Spark DataSource API.
- **Mode**: This is the Spark SaveMode, the Spark-HPCC Connector supports: *[ErrorIfExists, Ignore, Overwrite]*
    - Defaults to ErrorIfExists
- **Host**: The URL of an ESP running on the target HPCC Systems cluster.
- **Username / Password**: Credentials for an HPCC Systems cluster user, can be empty or null if security isn't enabled on the target cluster.
- **Cluster**: The name of the underlying Thor cluster storage plane, this will change based on the target HPCC Systems cluster configuration, but will default to "mythor" on bare-metal and "data" on containerized systems.
- **Path**: The file path for the dataset within the HPCC Systems cluster. **Note** Spark-HPCC versions [9.2.110, 9.4.84, 9.6.36, 9.8.10] and above allows for paths to be defined with "/" path delimiter instead of the HPCC "::" delimiter this fixes URI formatting errors on Databricks.
- **Compression**: The compression algorithm to use when writing the file to the HPCC Systems cluster.
    - Options: *[default, none, lz4, flz, lzw]*


In [ ]:
df.write.save(format="hpcc",
              mode="overwrite",
              host="http://127.0.0.1:8010",
              username="",
              password="",
              cluster="mythor",
              #path="spark::test::dataset", Old path format not supported on Databricks
              path="/spark/test/dataset",
              compression="default")

# Reading Data from HPCC Systems

---

A dataset from within an HPCC Systems cluster can be read via the Spark Datasource API.

- **Host**: The URL of an ESP running on the target HPCC Systems cluster.
- **Username / Password**: Credentials for an HPCC Systems cluster user, can be empty or null if security isn't enabled on the target cluster.
- **Cluster**: The name of the underlying Thor cluster storage plane, this will change based on the target HPCC Systems cluster configuration, but will default to "mythor" on bare-metal and "data" on containerized systems.
- **Path**: The file path for the dataset within the HPCC Systems cluster. **Note** Spark-HPCC versions [9.2.110, 9.4.84, 9.6.36, 9.8.10] and above allows for paths to be defined with "/" path delimiter instead of the HPCC "::" delimiter this fixes URI formatting errors on Databricks.
- **limitPerFilePart**: *Optional* Limit on the number of records to be read per file part / partition within the HPCC Systems dataset.
- **projectList**: *Optional* The columns that should be read from the HPCC Systems dataset.
- **useTLK** *Optional* Defaults to false, determines whether or not the TLK (Top Level Key) should be used when reading index files. 


In [ ]:
readDf = spark.read.load(format="hpcc",
                         host="http://127.0.0.1:8010",
                         username="",
                         password="",
                         useTLK="false",
                         cluster="mythor",
                         #path="spark::test::dataset", Old path format not supported on Databricks
                         path="/spark/test/dataset",
                         limitPerFilePart=100,
                         projectList="key, fill")

In [9]:
readDf.show()

+---+----------+
|key|      fill|
+---+----------+
|  0|8802828259|
|  1|6769815762|
|  2|3802883209|
|  3|7410875502|
|  4| 135941891|
|  5|2166775522|
|  6| 883758744|
|  7|3540920239|
|  8|  20735769|
|  9|4435630331|
| 10|2674809260|
| 11|2527618309|
| 12|9667830781|
| 13|8674197252|
| 14|2973016826|
| 15|7810363123|
| 16|3267854158|
| 17|9341494262|
| 18|8715303091|
| 19|7096204106|
+---+----------+
only showing top 20 rows

